In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances as ed
import evaluation

C:\Users\Wangzhc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


generate the relationship between aq_point and grid_point

In [2]:
loc_grid = pd.read_csv('Beijing_grid_weather_station.csv')

loc_aq = pd.read_csv('air_station.csv')
loc_ap = loc_aq.loc[:,['name','1','2']]

In [3]:
dic = {}
num = 1 # the num of nearest point to get weather information
for i in range(len(loc_ap)):
    name_aq = loc_aq.loc[i,['name']].values
    dic[name_aq[0]]=[]
    name_grid = loc_grid.loc[:,['name']].values
    loc_aq_ = loc_aq.loc[i,['1','2']].values.reshape(1,2)
    loc_grid_ = loc_grid.loc[:,['1','2']].values.reshape(-1,2)
    dis = ed(X=loc_grid_,Y=loc_aq_)
    sorted_dis = np.argsort(dis.reshape(1,-1))
    idx = list(sorted_dis[0,0:num])
    for j in range(num):
        dic[name_aq[0]].append(name_grid[idx[j],0])

read the data, this for small dataset for one month

In [5]:
grid = pd.read_csv('gridWeather_201804.csv')
observe =  pd.read_csv('observedWeather_201804.csv')
airq = pd.read_csv('aiqQuality_201804.csv')

read the data, this for small dataset for one year

In [21]:
airq = pd.read_csv('airQuality_201701-201801.csv')
grid = pd.read_csv('gridWeather_201701-201803.csv')

the column name is different , here is the name for year

In [24]:
cat_all = ['stationName','utc_time','temperature','pressure','humidity','wind_direction','wind_speed/kph']
cat = ['temperature','pressure','humidity','wind_direction','wind_speed/kph']
grid = grid.fillna(method="ffill")
grid = grid.loc[:,cat_all]
from sklearn.preprocessing import scale
for i in cat:
    grid[i] = grid[[i]].apply(scale)
#grid= pd.concat([grid,pd.get_dummies(grid['weather'])],axis=1)
#del grid['weather']

get attributes from weather grid

In [26]:
from tqdm import tqdm
data_dic={}
for i in tqdm(dic):
    t0 = airq[airq['stationId'] == i].sort_values(by=['utc_time'])
    #del t0['id']
    del t0['stationId']
    for j in dic[i]:
        t1 = grid[grid['stationName']== j]
        del t1['stationName']
        #del t1['id']
        t1.rename((lambda x: x+'_'+j if x != 'utc_time' else x) ,inplace=True,axis ='columns')
        t0 = t0.merge(t1,how='left',on=['utc_time'])
    del t0['utc_time']
    attr_num = len(t0.columns)
    t0 = t0.fillna(method="ffill")
    t0 = t0.dropna()
    data_dic[i] = (t0.values[0:-1,],t0.loc[:,['PM2.5', 'PM10','O3']].values[1:,])

  0%|          | 0/35 [00:00<?, ?it/s]C:\Users\Wangzhc\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
100%|██████████| 35/35 [00:31<00:00,  1.10it/s]


In [28]:
import numpy as np 
from matplotlib import pyplot 
import pandas as pd 
from keras.layers.core import Dense, Activation, Dropout 
from keras.models import Sequential 
import time 
import math 
from keras.layers import LSTM 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error 
from keras import optimizers 
import os
from keras.optimizers import SGD,Adam
from keras.layers.normalization import BatchNormalization as bn

train one lstm model for each station

In [ ]:
history = {}
for i in dic:
    print(i,'==============================================================================')
    attr_num = data_dic[i][0].shape[1]
    train_X,train_y=data_dic[i]
    train_X= train_X.reshape(-1,1,attr_num)
    train_y = train_y.reshape(-1,3)
    time_stamp = 10
    trainset={'x':[],'y':[]}
    valset={'x':[],'y':[]}
    import random
    for i in range(train_X.shape[0]-time_stamp+1):
        if random.random()<0.9:
            trainset['x'].append((train_X[i:i+time_stamp,:,:]).reshape(-1,time_stamp,attr_num))
            trainset['y'].append((train_y[i:i+time_stamp,:]).reshape(-1,time_stamp,3))
        else:
            valset['x'].append((train_X[i:i+time_stamp,:,:]).reshape(-1,time_stamp,attr_num))
            valset['y'].append((train_y[i:i+time_stamp,:]).reshape(-1,time_stamp,3))
    trainset['x'] = np.array(trainset['x']).reshape(-1,time_stamp,attr_num)
    valset['x'] = np.array(valset['x']).reshape(-1,time_stamp,attr_num)
    trainset['y'] = np.array(trainset['y']).reshape(-1,time_stamp,3)
    valset['y'] = np.array(valset['y']).reshape(-1,time_stamp,3)
    seq = not False
    unit = 32
    dropout=0.0
    model = Sequential()
    model.add(LSTM(unit,return_sequences=seq,dropout=dropout))
    model.add(bn())
    model.add(LSTM(unit//2,return_sequences=seq,dropout=dropout))
    model.add(bn())
    model.add(Dense(unit//4,activation='relu'))
    model.add(bn())
    model.add(Dense(3,activation='relu'))
    model.compile(loss='mse', optimizer='adam',metrics=[evaluation.smape])
    # fit network
    #[:,-1,:].reshape(-1,3)
    if not seq:
        history[i] = model.fit(trainset['x'], trainset['y'][:,-1,:].reshape(-1,3),validation_data=(valset['x'],valset['y'][:,-1,:].reshape(-1,3)), epochs=1500, batch_size=64,  verbose=2, shuffle=True)
    else:
        history[i] = model.fit(trainset['x'], trainset['y'],validation_data=(valset['x'],valset['y']), epochs=200, batch_size=64,  verbose=2, shuffle=True)


dongsi_aq ==============================================================================
Train on 8028 samples, validate on 848 samples
Epoch 1/200
 - 15s - loss: 10594.2935 - smape: 1.9286 - val_loss: 11407.7571 - val_smape: 1.9324
Epoch 2/200
 - 10s - loss: 10338.2076 - smape: 1.8981 - val_loss: 11146.5053 - val_smape: 1.9246
Epoch 3/200
 - 10s - loss: 10006.3078 - smape: 1.8702 - val_loss: 10897.9422 - val_smape: 1.9249
Epoch 4/200
 - 10s - loss: 9567.8540 - smape: 1.8390 - val_loss: 10858.0351 - val_smape: 1.9332
Epoch 5/200
 - 10s - loss: 9056.4551 - smape: 1.7889 - val_loss: 10087.4417 - val_smape: 1.8663
Epoch 6/200
 - 10s - loss: 8498.3601 - smape: 1.7219 - val_loss: 9298.9690 - val_smape: 1.7683
Epoch 7/200
 - 10s - loss: 7856.1055 - smape: 1.6312 - val_loss: 8512.5286 - val_smape: 1.6376
Epoch 8/200
 - 11s - loss: 7212.0444 - smape: 1.5252 - val_loss: 7909.2814 - val_smape: 1.5865
Epoch 9/200
 - 9s - loss: 6472.7147 - smape: 1.4120 - val_loss: 7358.7302 - val_smape: 1.4500
Ep

Epoch 87/200
 - 8s - loss: 430.1185 - smape: 0.4065 - val_loss: 510.0889 - val_smape: 0.3588
Epoch 88/200
 - 8s - loss: 441.2211 - smape: 0.4132 - val_loss: 387.5899 - val_smape: 0.3518
Epoch 89/200
 - 7s - loss: 423.6233 - smape: 0.4179 - val_loss: 424.0702 - val_smape: 0.3628
Epoch 90/200
 - 7s - loss: 442.0257 - smape: 0.4222 - val_loss: 451.2076 - val_smape: 0.3378
Epoch 91/200
 - 8s - loss: 420.3829 - smape: 0.4101 - val_loss: 422.3345 - val_smape: 0.3539
Epoch 92/200
 - 7s - loss: 410.3175 - smape: 0.4080 - val_loss: 393.2940 - val_smape: 0.3547
Epoch 93/200
 - 8s - loss: 413.6697 - smape: 0.4101 - val_loss: 388.8111 - val_smape: 0.3615
Epoch 94/200
 - 8s - loss: 406.6948 - smape: 0.4113 - val_loss: 388.1123 - val_smape: 0.3353
Epoch 95/200
 - 8s - loss: 415.8550 - smape: 0.4148 - val_loss: 611.6261 - val_smape: 0.4372
Epoch 96/200
 - 8s - loss: 417.6471 - smape: 0.4068 - val_loss: 388.7423 - val_smape: 0.3330
Epoch 97/200
 - 8s - loss: 411.2356 - smape: 0.4024 - val_loss: 402.10

Epoch 175/200
 - 8s - loss: 370.9567 - smape: 0.4087 - val_loss: 360.6646 - val_smape: 0.3627
Epoch 176/200
 - 8s - loss: 379.3756 - smape: 0.4121 - val_loss: 349.7385 - val_smape: 0.3347
Epoch 177/200
 - 8s - loss: 370.4472 - smape: 0.4074 - val_loss: 383.3011 - val_smape: 0.3482
Epoch 178/200
 - 8s - loss: 386.3080 - smape: 0.4157 - val_loss: 345.5365 - val_smape: 0.3845
Epoch 179/200
 - 8s - loss: 376.6167 - smape: 0.4139 - val_loss: 381.8777 - val_smape: 0.3566
Epoch 180/200
 - 8s - loss: 376.7920 - smape: 0.4118 - val_loss: 357.0993 - val_smape: 0.3623
Epoch 181/200
 - 7s - loss: 373.4593 - smape: 0.4148 - val_loss: 355.2357 - val_smape: 0.3513
Epoch 182/200
 - 8s - loss: 364.1974 - smape: 0.4083 - val_loss: 364.8081 - val_smape: 0.3589
Epoch 183/200
 - 8s - loss: 378.7551 - smape: 0.4107 - val_loss: 441.5447 - val_smape: 0.3588
Epoch 184/200
 - 8s - loss: 363.8771 - smape: 0.4086 - val_loss: 364.8594 - val_smape: 0.3372
Epoch 185/200
 - 8s - loss: 381.1930 - smape: 0.4181 - val_l

Epoch 61/200
 - 11s - loss: 528.1195 - smape: 0.5026 - val_loss: 523.3869 - val_smape: 0.5427
Epoch 62/200
 - 11s - loss: 525.9201 - smape: 0.5062 - val_loss: 452.8187 - val_smape: 0.4810
Epoch 63/200
 - 11s - loss: 547.2902 - smape: 0.5055 - val_loss: 485.4682 - val_smape: 0.4668
Epoch 64/200
 - 11s - loss: 508.9710 - smape: 0.4896 - val_loss: 418.0748 - val_smape: 0.4431
Epoch 65/200
 - 11s - loss: 521.8408 - smape: 0.4968 - val_loss: 413.9793 - val_smape: 0.4524
Epoch 66/200
 - 11s - loss: 517.2672 - smape: 0.4950 - val_loss: 398.6394 - val_smape: 0.4359
Epoch 67/200
 - 11s - loss: 507.5444 - smape: 0.4976 - val_loss: 377.4497 - val_smape: 0.4349
Epoch 68/200
 - 11s - loss: 528.7123 - smape: 0.5019 - val_loss: 501.1483 - val_smape: 0.4520
Epoch 69/200
 - 11s - loss: 528.2105 - smape: 0.4901 - val_loss: 411.4209 - val_smape: 0.4713
Epoch 70/200
 - 11s - loss: 515.3171 - smape: 0.4932 - val_loss: 379.3193 - val_smape: 0.4402
Epoch 71/200
 - 11s - loss: 503.9582 - smape: 0.4904 - val_l

Epoch 148/200
 - 9s - loss: 470.1146 - smape: 0.4754 - val_loss: 687.7838 - val_smape: 0.4515
Epoch 149/200
 - 9s - loss: 428.4750 - smape: 0.4665 - val_loss: 358.7543 - val_smape: 0.4100
Epoch 150/200
 - 9s - loss: 429.2360 - smape: 0.4656 - val_loss: 394.2280 - val_smape: 0.4345
Epoch 151/200
 - 9s - loss: 439.5444 - smape: 0.4718 - val_loss: 340.1334 - val_smape: 0.4085
Epoch 152/200
 - 9s - loss: 434.6316 - smape: 0.4624 - val_loss: 365.6458 - val_smape: 0.4309
Epoch 153/200
 - 9s - loss: 425.2185 - smape: 0.4728 - val_loss: 355.9910 - val_smape: 0.4202
Epoch 154/200
 - 9s - loss: 394.9222 - smape: 0.4588 - val_loss: 419.4678 - val_smape: 0.4464
Epoch 155/200
 - 9s - loss: 411.9797 - smape: 0.4655 - val_loss: 372.7433 - val_smape: 0.4248
Epoch 156/200
 - 9s - loss: 411.6300 - smape: 0.4614 - val_loss: 367.7872 - val_smape: 0.4162
Epoch 157/200
 - 9s - loss: 425.3675 - smape: 0.4665 - val_loss: 353.1997 - val_smape: 0.4430
Epoch 158/200
 - 9s - loss: 407.5151 - smape: 0.4573 - val_l

In [120]:
# plot history
for i in dic:
    
    pyplot.plot(history.history[i]['loss'], label='train')
    pyplot.plot(history.history[i]['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()


In [121]:
valset['x'].shape

(843, 10, 11)